In [1]:
# List values or ranges for parameters
import random
import numpy as np

prelim = {"Vin": [200],
          "Vref": (160, 160),
          "P": (50, 1100),
          "D1": np.arange(0.6, 1.01, 0.01).round(2).tolist(),
          "D2": [1.0],
         }

def get_a_param_set(prelim):
    param_set = {}
    for key in prelim:
        if isinstance(prelim[key], tuple):
            # indicate a range
            value = round(np.random.uniform(*prelim[key]), 1)
        elif isinstance(prelim[key], list):
            # indicate selection
            value = np.random.choice(prelim[key])
        param_set[key] = float(value)
    return param_set

In [ ]:
import threading
import numpy as np
import xmlrpc.client
import os
import pandas as pd
import csv
import time
import logging
import os
import shutil
import func_timeout
from func_timeout import func_set_timeout
from tqdm import tqdm


class PlecsThread(threading.Thread):
    
    opts = {'ModelVars': {'D1': 0.85, 'D2': 0.55, 'Vref': 200, 'P': 100, 'Ro': 200**2/100}}
    
    def __init__(self, threadID, name, model_name, result_path, num_req=1e4):
        super(PlecsThread, self).__init__()
        self.threadID = threadID
        self.name = name
        self.model_name = model_name
        self.num_req = num_req
        self.result_path = result_path + f"\\Performance.csv"
        self.result_path2 = result_path + f"\\Waveform"
        self.model_path = os.getcwd() + f"\\{self.model_name}.plecs"
        self.to_file()
        self.server = xmlrpc.client.Server('http://localhost:1080/RPC2"')
        
    def to_file(self):
        if not os.path.exists(self.result_path):
            columns = "idx Vin Vout Iload Duty_Cycle Switching_Frequency On_resistance Gate_charge Total_Gate_Resistance Inductance Series Resistance Cap"
            df = pd.DataFrame(columns=columns.split())
            df.to_csv(self.result_path, index=False)
            
    def load(self):
        self.server.plecs.load(self.model_path)
    
    def close(self):
        self.server.plecs.close(self.model_name)
    
    @func_set_timeout(60*3)
    def run_sim(self, opts):
        self.load()
        self.server.plecs.simulate(self.model_name, opts)
        self.close()
        
    def run(self):
        for i in tqdm(range(self.num_req)):
            self.loop(i)
    
    def loop(self, i):
        try:
            opts0 = get_a_param_set(prelim)
            opts = {'ModelVars': {}}
            opts['ModelVars'].update(opts0)
            opts['ModelVars']["Ro"] = round(opts0["Vref"]**2/opts0["P"], 2)
            print(opts)
            self.run_sim(opts)
            df = pd.read_csv(os.getcwd() + f"\\Single_Sim{self.threadID}.csv", header=None)
            with open(self.result_path, 'a', newline='') as f: 
                csv.writer(f).writerow(np.hstack(([i], list(get_a_param_set(prelim).values()), 
                                                  df.iloc[-1, 1:].values)))
            shutil.move("Single_Sim1_Wave.csv", os.path.join(self.result_path2, f"{i}.csv"))
        except Exception as e:
            print(e)

In [ ]:
import pandas as pd
import datetime

threads_num = 1
result_path = "./Buck_data"
num_req = 100

start_time = datetime.datetime.now()
threads = []
for i in range(threads_num):
    thread = PlecsThread(i+1, f"{i+1}", f"DAB_sample{i+1}", result_path, num_req)
    threads.append(thread)
    thread.start()
for i in range(threads_num):
    threads[i].join()


<>:5: SyntaxWarning: invalid escape sequence '\B'
<>:5: SyntaxWarning: invalid escape sequence '\B'
C:\Users\ssefa\AppData\Local\Temp\ipykernel_69424\308285201.py:5: SyntaxWarning: invalid escape sequence '\B'
  result_path = ".\Buck_data"
C:\Users\ssefa\AppData\Local\Temp\ipykernel_69424\308285201.py:5: SyntaxWarning: invalid escape sequence '\B'
  result_path = ".\Buck_data"


NameError: name 'PlecsThread' is not defined

Fault: <Fault -32500: "method execution error: Model 'Buck_sample1' already loaded from different location.">